In [1]:
import codecs
import string
from nltk.tokenize import sent_tokenize, word_tokenize
import re, os, operator
import pickle
from matplotlib import pyplot as plt
from random import randint as randi
from math import log10
import numpy as np
from tqdm import tqdm_notebook as tqdm
from random import randint as randi
import copy

In [2]:
def getTokens(fileName):
    f = open(fileName, 'r')
    text = PreProcessing(f.read())
    tokens = word_tokenize(text)
    f.close()
    return tokens

def getTestingTokens(sent):
    text = PreProcessing(sent, False)
    tokens = word_tokenize(text)
    return tokens

def getSentenceTokens(fileName):
    f = open(fileName, 'r')
    text = SentPreProcessing(f.read())
    tokens = sent_tokenize(text)
#     text = re.sub(r'[^\w]', ' ', text)
    f.close()
    return tokens

def SentPreProcessing(text):
    header = text.find("\n\n")
    header = text.find("\n\n", header+4)
    text = text[header:]
    text = text.lower()
    text = re.sub(r'[\w\$\.-]+@[\w\.-]+\.\w+', ' ', text)
#     for i in string.punctuation:
#         cmd = '\b' + str(i) + '\b'
#         text = text.replace(cmd, '\b\b')
    return text

def PreProcessing(text, flag=True):
    if flag:
        header = text.find("\n\n")
        header = text.find("\n\n", header+4)
        text = text[header:]
    text = text.strip()
    text = text.rstrip()
    text = text.lower()
    text = re.sub(r'[\w\$\.-]+@[\w\.-]+\.\w+', ' ', text)
#     text = re.sub()
    for i in string.punctuation:
        text = text.replace(i, '')
#     for i in string.punctuation:
#         cmd = '\b' + str(i) + '\b'
#         text = text.replace(cmd, '\b\b')
#     text = re.sub(r'[0-9]', '', text)
    text = re.sub(r'[^\w]', ' ', text)
    return text

def getUniGramCount(tokens, uniDic):
    uni = uniDic
    for tkn in tokens:
        if tkn in uni:
            uni[tkn] += 1
        else:
            uni[tkn] = 1
    return uni, len(tokens)

def getBiGramCount(tokens, bidic):
    BiGramTable = bidic
    BigramCount = 0
    for i in range(len(tokens)-1):
        word = tokens[i]
        nextWord = tokens[i+1]
        if word in BiGramTable:
            if nextWord in BiGramTable[word]:
                BiGramTable[word][nextWord] += 1
            else:
                BiGramTable[word][nextWord] = 1
                BigramCount += 1
        else:
            BiGramTable[word] = {}
            BiGramTable[word][nextWord] = 1
            BigramCount += 1
    return BiGramTable, BigramCount

def getTriGramCount(tokens, tridic):
    TriGramTable = tridic
    TrigramCount = 0
    for i in range(len(tokens)-2):
        word = tokens[i]
        nextWord = tokens[i+1]
        nextnextWord = tokens[i+2]
        if word in TriGramTable:
            if nextWord in TriGramTable[word]:
                if nextnextWord in TriGramTable[word][nextWord]:
                    TriGramTable[word][nextWord][nextnextWord] += 1
                else:
                    TriGramTable[word][nextWord][nextnextWord] = 1
                    TrigramCount += 1
            else:
                TriGramTable[word][nextWord] = {}
                TriGramTable[word][nextWord][nextnextWord] = 1
                TrigramCount += 1
        else:
            TriGramTable[word] = {}
            TriGramTable[word][nextWord] = {}
            TriGramTable[word][nextWord][nextnextWord] = 1
            TrigramCount += 1
    return TriGramTable, TrigramCount

def predictUniProbability(tokens, UniArr, tokensAll, priors):
    totalTokens = tokensAll[0] + tokensAll[1]
    Vocab = list(set(totalTokens))
    probs = np.zeros(2)
    for j in range(2):
        proba = 0
        for i in range(len(tokens)):
            if tokens[i] in UniArr[j]:
                proba += log10((UniArr[j][tokens[i]] + 1)/(len(tokensAll[j]) + len(Vocab)))
            else:
                proba += log10(1/(len(tokensAll[j]) + len(Vocab)))
        probs[j] = proba
    return probs

def predictBiProbability(tokens, UniArr, BiArr, tokensAll, priors):
    tokens = ["<Start>"] + tokens
    totalTokens = tokensAll[0] + tokensAll[1]
    Vocab = list(set(totalTokens))
    probs = np.zeros(2)
    for j in range(2):
        proba = 0
        for i in range(len(tokens)-1):
            word = tokens[i]
            nextWord = tokens[i+1]
            if word in BiArr[j]:
                if nextWord in BiArr[j][word]:
                    proba += log10((BiArr[j][word][nextWord] + 1)/(UniArr[j][word] + len(Vocab)))
                else:
                    proba += log10(1/(UniArr[j][word] + len(Vocab)))
            else:
                proba += log10(1/(len(Vocab)))
        probs[j] = proba
    return probs

def predictTriProbability(tokens, BiArr, TriArr, tokensAll, priors):
    tokens = ["<Start>"] + tokens
    totalTokens = tokensAll[0] + tokensAll[1]
    Vocab = list(set(totalTokens))
    probs = np.zeros(2)
    for j in range(2):
        proba = 0
        for i in range(len(tokens)-2):
            word = tokens[i]
            nextWord = tokens[i+1]
            nextnextWord = tokens[i+2]
            if word in TriArr[j]:
                if nextWord in TriArr[j][word]:
                    if nextnextWord in TriArr[j][word][nextWord]:
                        proba += log10((1 + TriArr[j][word][nextWord][nextnextWord])/ (BiArr[j][word][nextWord] + len(Vocab)))
                    else:
                        proba += log10(1/(BiArr[j][word][nextWord] + len(Vocab)))
                else:
                    proba += log10(1/len(Vocab))
            else:
                proba += log10(1/len(Vocab))
        probs[j] = proba
    return probs

def getUniSentence(UniArr, tokensAll, priors, MinWordLength=5, MaxWordLength=9):
    totalTokens = tokensAll[0] + tokensAll[1]
    Vocab = list(set(totalTokens))
    for i in range(2):
        SortedUniDic = sorted(UniArr[i].items(), key=lambda kv: kv[1], reverse=True)
        proba = 0
        index = 0
        Keys = []
        for key, value in SortedUniDic:
            if (key != "<Start>"):
                Keys.append(key)
        Sentence = []
#         print(len(Keys), Keys)
        while(len(Sentence) < MaxWordLength):
            temp = log10((UniArr[i][Keys[index]] + 1)/(len(tokensAll) + len(Vocab)))
            if (len(Sentence) > MinWordLength and (temp + proba) < proba ):
                break
            if ((Keys[index] == "." or Keys[index] == "!" or Keys[index] == "?") and len(Sentence) > MinWordLength):
                Sentence.append(Keys[index])
                proba += temp
                break
            if (Keys[index] == "." or Keys[index] == "!" or Keys[index] == "?"):
                index += 1
                continue
            Sentence.append(Keys[index])
            proba += temp
            index += 1
        print("Sentence Generation using UniGram Methods [Class {}]".format(i+1), end= ": ")
        for w in Sentence:
            print(w, end=" ")
        print("\n")
        print(proba)
        print()
        
def getNextWord(prevWord, bidic):    
    Keys = bidic[prevWord].keys()
    MaxWord = ""
    count = 0
    for k in Keys:
        if bidic[prevWord][k] > count:
            count = bidic[prevWord][k]
            MaxWord = k
    return MaxWord
    
    
def getBiSentence(BiArr, UniArr, tokensAll, MinWordLength=5, MaxWordLength=9):
    totalTokens = tokensAll[0] + tokensAll[1]
    Vocab = list(set(totalTokens))
    for i in range(2):
        proba = 0
        Sentence = []
        prevWord = "<Start>"
        while(len(Sentence) < MaxWordLength):
            word = getNextWord(prevWord, BiArr[i])
            temp = log10((BiArr[i][prevWord][word] + 1)/(UniArr[i][prevWord] + len(Vocab)))
            if (len(Sentence) > MinWordLength and (temp + proba) < proba ):
                break
            if ((word == "." or word == '?' or word == "!") and len(Sentence) > MinWordLength):
                Sentence.append(word)
                proba += temp
                break
            Sentence.append(word)
            proba += temp
            prevWord = word
        print("Sentence Generation using BiGram Methods [Class {}]".format(i+1), end= ": ")
        for w in Sentence:
            print(w, end=" ")
        print("\n")
        print(proba)
        print()   
    
def getTriSentence(TriArr, BiArr, tokensAll, MinWordLength=5, MaxWordLength=9):
    totalTokens = tokensAll[0] + tokensAll[1]
    Vocab = list(set(totalTokens))
    for i in range(2):
        proba = 0
        Sentence = []
        word = "<Start>"
        nextWord, nextnextWord = getInitialWords(TriArr[i])
        Sentence.append(nextWord)
        Sentence.append(nextnextWord)
        while(len(Sentence) < MaxWordLength):
            nextnextnextWord = getNextWordTri(nextWord, nextnextWord, TriArr[i])
            temp = log10((TriArr[i][nextWord][nextnextWord][nextnextnextWord] + 1)/(BiArr[i][nextWord][nextnextWord] + len(Vocab)))
            if (len(Sentence) > MinWordLength and (temp + proba) < proba ):
                break
            if ((word == "." or word == '?' or word == "!") and len(Sentence) > MinWordLength):
                Sentence.append(word)
                proba += temp
                break
            Sentence.append(nextnextnextWord)
            nextWord = nextnextWord
            nextnextWord = nextnextnextWord
            proba += temp
        print("Sentence Generation using TriGram Methods [Class {}]".format(i+1), end= ": ")
        for w in Sentence:
            print(w, end=" ")
        print("\n")
        print(proba)
        print()   
        
def getNextWordTri(w1, w2, tridic):
    count = 0
    word = ""
    for k in tridic[w1][w2].keys():
        if (count < tridic[w1][w2][k]):
            count = tridic[w1][w2][k]
            word = k
    return word
        
def getInitialWords(tridic):
    SW = "<Start>"
    nextWord = ""
    nextnextWord = ""
    count = 0
    for k in tridic[SW].keys():
        for k2 in tridic[SW][k].keys():
            if tridic[SW][k][k2] > count:
                count = tridic[SW][k][k2]
                nextWord = k
                nextnextWord = k2
    return nextWord, nextnextWord
        
def ConsiderUniStartWord(unidic, sent):
    SW = "<Start>"
    if SW not in unidic:
        unidic[SW] = len(sent)
    unidic[SW] += len(sent)
    return unidic

def ConsiderBiStartWord(bidic, sent):
    SW = "<Start>"
    if SW not in bidic:
        bidic[SW] = {}
    for i in range(len(sent)):
        s = sent[i]
        words = word_tokenize(s)
        if (len(words) < 1):
            continue
        if not re.match("^[a-zA-Z0-9_]*$", words[0]):
            continue
        if words[0] in bidic["<Start>"]:
            bidic["<Start>"][words[0]] += 1
        else:
            bidic["<Start>"][words[0]] = 1
        
#         for j in range(len(words)-1):
#             w = words[j]
#             if w in BiDic["<Start>"]:
#                 BiDic["<Start>"][w] += 1
#             else:
#                 BiDic["<Start>"][w] = 1
    return bidic

def ConsiderTriStartWord(tridic, sent):
    SW = "<Start>"
    if SW not in tridic:
        tridic[SW] = {}
    for i in range(len(sent)):
        s = sent[i]
        words = word_tokenize(s)
        if (len(words) < 2):
            continue
        if ((not re.match("^[a-zA-Z0-9_]*$", words[0])) or (not re.match("^[a-zA-Z0-9_]*$", words[1]))):
            continue
        w1 = words[0]
        w2 = words[1]
        if w1 in tridic["<Start>"]:
            if w2 in tridic["<Start>"][w1]:
                tridic["<Start>"][w1][w2] += 1
            else:
                tridic["<Start>"][w1][w2] = 1
        else:
            tridic["<Start>"][w1] = {}
            tridic["<Start>"][w1][w2] = 1
#         for j in range(len(words)-2):
#             w = words[j]
#             nextW = words[j+1]
#             if w in TriDic["<Start>"]:
#                 if nextW in TriDic["<Start>"][w]:
#                     TriDic["<Start>"][w][nextW] += 1
#                 else:
#                     TriDic["<Start>"][w][nextW] = 1
#             else:
#                 TriDic["<Start>"][w] = {}
#                 TriDic["<Start>"][w][nextW] = 1
    return tridic

## Training

In [3]:
UniArr = []
BiArr = []
TriArr = []
TokensAll = []
priors = np.zeros(2)
Class = ["20_newsgroups/rec.sport.baseball/", "20_newsgroups/rec.motorcycles/"]
for i in range(2):
    
    foldersName = os.listdir(Class[i])
    tokens = []
    UniDic = {}
    BiDic = {}
    TriDic = {}
    priors[i] = len(foldersName)
    for j in range(len(foldersName)):
        tkn = getTokens(Class[i] + foldersName[j])
        sent = getSentenceTokens(Class[i] + foldersName[j])
        UniDic = ConsiderUniStartWord(UniDic, sent)
        BiDic = ConsiderBiStartWord(BiDic, sent)
        TriDic = ConsiderTriStartWord(TriDic, sent)
        UniDic, uni_count = getUniGramCount(tkn, UniDic)
        BiDic, bi_count = getBiGramCount(tkn, BiDic)
        TriDic, tri_count = getTriGramCount(tkn, TriDic)
        tokens += tkn
        
    TokensAll.append(tokens)
    UniArr.append(UniDic)
    BiArr.append(BiDic)
    TriArr.append(TriDic)
#     print(TriDic)
priors /= (priors[0]+priors[1])
print(priors)

[0.5 0.5]


## Sentence Generation

In [4]:
getUniSentence(UniArr, TokensAll, priors)
getBiSentence(BiArr, UniArr, TokensAll)
getTriSentence(TriArr, BiArr, TokensAll)

Sentence Generation using UniGram Methods [Class 1]: the to a and of in 

-4.79695062760876

Sentence Generation using UniGram Methods [Class 2]: the a to i and of 

-4.980958775324477

Sentence Generation using BiGram Methods [Class 1]: i think that the game in 

-13.272894857703374

Sentence Generation using BiGram Methods [Class 2]: i was a few weeks i 

-15.074211353175436

Sentence Generation using TriGram Methods [Class 1]: i do not know what it 

-13.203913269137793

Sentence Generation using TriGram Methods [Class 2]: if you want to go left 

-13.029959819975991



## Predicting probability And Perplexity

In [24]:
sentence = str(input("Enter a sentence: "))
tokens = getTestingTokens(sentence)

uniProb = predictUniProbability(tokens, UniArr, TokensAll, priors)
print("UniGram Probability:", uniProb)

biProb = predictBiProbability(tokens, UniArr, BiArr, TokensAll, priors)
print("BiGram Probability:", biProb)

triProb = predictTriProbability(tokens, BiArr, TriArr, TokensAll, priors)
print("TriGram Probability:", triProb)

print()

for i in range(2):
    uniProb[i] = uniProb[i] * (-1/len(tokens))
    uniProb[i] = 10**uniProb[i]
print("UniGram Perplexity:", uniProb)

for i in range(2):
    biProb[i] = biProb[i] * (-1/len(tokens))
    biProb[i] = 10**biProb[i]
print("BiGram Perplexity:", biProb)

for i in range(2):
    triProb[i] = triProb[i]*(-1/len(tokens))
    triProb[i] = 10**triProb[i]
print("TriGram Perplexity:", triProb)

Enter a sentence: How about a Geeky temporary tatoo?
UniGram Probability: [-22.81233518 -20.54741237]
BiGram Probability: [-21.95600977 -20.39957009]
TriGram Probability: [-20.36166876 -18.38628068]

UniGram Perplexity: [6339.51251297 2658.08416097]
BiGram Perplexity: [4563.88783333 2511.47204032]
TriGram Perplexity: [2475.2066304  1159.79201026]


## Good Turing Smoothing

In [5]:
def UniGTSmoothing(UniArr):
    NUni = []
    for i in range(2):
        count = np.ones(10000, dtype=np.int64)
        for w in UniArr[i]:
            count[UniArr[i][w]] += 1
        NUni.append(count)
    return NUni

def BiGTSmoothing(BiArr):
    NBi = []
    for i in range(2):
        count = np.ones(10000, dtype=np.int64)
        for w in BiArr[i]:
            for w2 in BiArr[i][w]:
                count[BiArr[i][w][w2]] += 1
        NBi.append(count)
    return NBi

def TriGTSmoothing(TriArr):
    NTri = []
    for i in range(2):
        count = np.ones(10000, dtype=np.int64)
        for w in TriArr[i]:
            for w2 in TriArr[i][w]:
                for w3 in TriArr[i][w][w2]:
                    count[TriArr[i][w][w2][w3]] += 1
        NTri.append(count)
    return NTri

def predictUniProbGT(tokens, UniArr, tokensAll, NUni):
    totalTokens = tokensAll[0] + tokensAll[1]
    Vocab = list(set(totalTokens))
    probs = np.zeros(2)
    for j in range(2):
        proba = 0
        for i in range(len(tokens)):
            if tokens[i] in UniArr[j]:
                c = UniArr[j][tokens[i]] + 1
                cStar = (c*NUni[j][c])/ NUni[j][c-1]
                proba += log10(cStar/len(tokensAll[j]))
            else:
                proba += log10(NUni[j][1]/len(tokensAll[j]))
#         print(probs)
        probs[j] = proba
    return probs

def predictBiProbGT(tokens, UniArr, BiArr, tokensAll, NBi, NUni):
    tokens = ["<Start>"] + tokens
    totalTokens = tokensAll[0] + tokensAll[1]
    Vocab = list(set(totalTokens))
    probs = np.zeros(2)
    for j in range(2):
        proba = 0
        for i in range(len(tokens)-1):
            word = tokens[i]
            nextWord = tokens[i+1]
            if word in BiArr[j]:
                if nextWord in BiArr[j][word]:
                    c = BiArr[j][word][nextWord] + 1
                    cStar = (c*NBi[j][c])/ (NBi[j][c-1]*len(tokensAll))
                    proba += log10(cStar)
                else:
                    proba += log10(NBi[j][1]/len(tokensAll))
        probs[j] = proba
    return probs

def predictTriProbGT(tokens, BiArr, TriArr, tokensAll, NTri, NBi):
    tokens = ["<Start>"] + tokens
    totalTokens = tokensAll[0] + tokensAll[1]
    Vocab = list(set(totalTokens))
    probs = np.zeros(2)
    for j in range(2):
        proba = 0
        for i in range(len(tokens)-2):
            word = tokens[i]
            nextWord = tokens[i+1]
            nextnextWord = tokens[i+2]
            if word in TriArr[j]:
                if nextWord in TriArr[j][word]:
                    if nextnextWord in TriArr[j][word][nextWord]:
                        c = TriArr[j][word][nextWord][nextnextWord] + 1
                        cStar = (c*NTri[j][c])/(NTri[j][c-1]*len(tokensAll))
                        proba += log10(cStar)
                    else:
                        proba += log10(NTri[j][1]/len(tokensAll))
        probs[j] = proba
    return probs

In [6]:
NUni = UniGTSmoothing(UniArr)
NBi = BiGTSmoothing(BiArr)
NTri = TriGTSmoothing(TriArr)

In [7]:
sentence = str(input("Enter a sentence: "))
tokens = getTestingTokens(sentence)

uniProb = predictUniProbGT(tokens, UniArr, TokensAll, NUni)
print("UniGram Probability:", uniProb)

biProb = predictBiProbGT(tokens, UniArr, BiArr, TokensAll, NBi, NUni)
print("BiGram Probability:", biProb)

triProb = predictTriProbGT(tokens, BiArr, TriArr, TokensAll, NTri, NBi)
print("TriGram Probability:", triProb)

Enter a sentence: the sale of the Orioles to anyone is likely
UniGram Probability: [-25.08234787 -21.39245556]
BiGram Probability: [18.12941897 16.02243749]
TriGram Probability: [23.27938822 18.31311241]


In [ ]:
# UniArrGT = []
# BiArrGT = []
# TriArrGT = []
# for i in range(2):
#     UniArrGT.append(copy.deepcopy(UniArr[i]))
#     BiArrGT.append(copy.deepcopy(BiArr[i]))
#     TriArrGT.append(copy.deepcopy(TriArr[i]))